In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import pickle
import torch
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [3]:
cuda_available = torch.cuda.is_available()
print(f'CUDA Available: {cuda_available}')
if cuda_available:
    print('CUDA device:', torch.cuda.get_device_name(0))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

CUDA Available: True
CUDA device: Tesla T4
device: cuda


# Running Model and getting output

READING IN DATA

In [4]:
#Matt

#matt_dzwil
#df = pd.read_pickle('/content/drive/MyDrive/Home Projects/Big Data Bowl 2023/data/model_input_200k_random_sample.pkl')
#data = pd.read_pickle('/content/drive/MyDrive/Home Projects/Big Data Bowl 2023/data/with_play_data_250k.pkl')
#dzwil_matt
#data = pd.read_pickle('/content/drive/MyDrive/Big Data Bowl 2023/data/with_play_data_250k.pkl')
data = pd.read_pickle('/content/drive/MyDrive/Big Data Bowl 2023/data/with_play_data_250k_testing.pkl')
#data = pd.read_pickle('/content/drive/MyDrive/Big Data Bowl 2023/data/model_input_200k_random_sample.pkl')

In [ ]:
#Katie

df = pd.read_pickle('/content/drive/MyDrive/model_input_no_id_all.pkl')
df.columns.to_list()

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Big Data Bowl 2023/model_input_no_id_100k.csv')
df.columns.to_list()

In [ ]:
# data = df.copy()
#data = data.iloc[:, 1:]
data

,gameId,playId,frameId,four,three,two,one,current,tackle_1,tackle_2,...,quarter,down,yardsToGo,yardlineNumber,totalSeconds,possesionScore,defensiveScore,absoluteYardlineNumber,possessionWinProb,defensiveWinProb
0,2022090800,56,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.9823384598581234, 1.3484585884884548, -0.13...",0.0,1.0,...,1,1,10,25,900,0,0,85,0.586653,0.413347
1,2022090800,56,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.9823384598581234, 1.3484585884884548, -0.13...","[0.790808421685418, 1.4283941077210496, -0.733...",0.0,1.0,...,1,1,10,25,900,0,0,85,0.586653,0.413347
2,2022090800,56,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.9823384598581234, 1.3484585884884548, -0.13...","[0.790808421685418, 1.4283941077210496, -0.733...","[0.5098976990321165, 1.656781305528464, -0.866...",0.0,1.0,...,1,1,10,25,900,0,0,85,0.586653,0.413347
3,2022090800,56,4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.9823384598581234, 1.3484585884884548, -0.13...","[0.790808421685418, 1.4283941077210496, -0.733...","[0.5098976990321165, 1.656781305528464, -0.866...","[0.07576294584065033, 1.668200665418835, -0.42...",0.0,1.0,...,1,1,10,25,900,0,0,85,0.586653,0.413347
4,2022090800,56,5,"[0.07576294584065033, 1.668200665418835, -0.42...","[0.5098976990321165, 1.656781305528464, -0.866...","[0.790808421685418, 1.4283941077210496, -0.733...","[0.9823384598581234, 1.3484585884884548, -0.13...","[-0.26899112287021976, 1.8965878632262487, -0....",0.0,1.0,...,1,1,10,25,900,0,0,85,0.586653,0.413347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262019,2022110700,3686,72,"[-0.03915507706297314, 0.4349097972587982, -0....","[0.011919599783081736, 0.5262646763817639, -0....","[0.0629942766291366, 0.7203937945180658, -0.98...","[-0.0136177386399457, 0.8916841928736267, -1.0...","[-0.24345378444719232, 0.21794195934175478, -0...",0.0,0.0,...,4,1,10,46,151,27,13,64,0.996820,0.003180
262020,2022110700,3686,73,"[-0.24345378444719232, 0.21794195934175478, -0...","[-0.03915507706297314, 0.4349097972587982, -0....","[0.011919599783081736, 0.5262646763817639, -0....","[0.0629942766291366, 0.7203937945180658, -0.98...","[-0.5499018455235213, 0.09232900054767713, -0....",0.0,0.0,...,4,1,10,46,151,27,13,64,0.996820,0.003180
262021,2022110700,3686,74,"[-0.5499018455235213, 0.09232900054767713, -0....","[-0.24345378444719232, 0.21794195934175478, -0...","[-0.03915507706297314, 0.4349097972587982, -0....","[0.011919599783081736, 0.5262646763817639, -0....","[-0.652051199215631, 0.0009741214247113909, -0...",0.0,0.0,...,4,1,10,46,151,27,13,64,0.996820,0.003180
262022,2022110700,3686,75,"[-0.652051199215631, 0.0009741214247113909, -0...","[-0.5499018455235213, 0.09232900054767713, -0....","[-0.24345378444719232, 0.21794195934175478, -0...","[-0.03915507706297314, 0.4349097972587982, -0....","[-0.7158945452731994, -0.10180011758862507, -0...",0.0,0.0,...,4,1,10,46,151,27,13,64,0.996820,0.003180


In [5]:
def add_value_to_array(arr):
    if len(arr) == 175:
        return np.append(arr, 0.0)
    else:
        return arr

def divide_game_id(integer):
    string = str(integer)
    return int(string[4:])

data['four'] = data['four'].apply(add_value_to_array)
data['three'] = data['three'].apply(add_value_to_array)
data['two'] = data['two'].apply(add_value_to_array)
data['one'] = data['one'].apply(add_value_to_array)
data['gameId'] = data['gameId'].apply(divide_game_id)

In [6]:
data.columns

Index(['gameId', 'playId', 'frameId', 'four', 'three', 'two', 'one', 'current',
       'tackle_1', 'tackle_2', 'tackle_3', 'tackle_4', 'tackle_5', 'tackle_6',
       'tackle_7', 'tackle_8', 'tackle_9', 'tackle_10', 'tackle_11',
       'tackle_12', 'quarter', 'down', 'yardsToGo', 'yardlineNumber',
       'totalSeconds', 'possesionScore', 'defensiveScore',
       'absoluteYardlineNumber', 'possessionWinProb', 'defensiveWinProb'],
      dtype='object')

In [7]:
unique_combinations = len(data[data['playId']> 900])
unique_combinations


199959

In [ ]:
from sklearn.model_selection import GroupShuffleSplit

X = data.iloc[:, :-12]
y = data.iloc[:, -12:]

# Define the groups
groups = data['playId']

# Initialize the splitter
# n_splits: The number of re-shuffling & splitting iterations. For a simple train/test split, this is 1.
gss = GroupShuffleSplit(n_splits=1, test_size=0.25, random_state=41)

# Split the data ensuring the same group is not in both training and testing sets

#Then, it divides these groups into training and test sets, ensuring no group is split between the two.
# This means all entries from a particular 'playId' will be entirely in the training set or entirely in the test set.
for train_idx, test_idx in gss.split(X, y, groups):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

# Now you can proceed to use X_train, X_test, y_train, y_test for training and testing your model


TO DO: NEED TO FIGURE OUT WAY TO SPLIT BASED ON PLAY

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, :-12], data.iloc[:, -12:], test_size=0.25, random_state=41)
train = data[data['playId'] > 900]
test = data[data['playId'] <= 900]

In [8]:
import torch
from torch.utils.data import Dataset

class IntitalModelDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
      tensor_list = []
      for i in range(5):
        #print(type(self.dataframe.iloc[idx, i+3]))
        column_tensor = torch.tensor(self.dataframe.iloc[idx, i+3])

        tensor_list.append(column_tensor)
      result_tensor = torch.stack(tensor_list)
      # print("WHAT IT Should Be",(self.dataframe.iloc[idx, :3]))
      # print("WHAT IT IS",torch.FloatTensor(self.dataframe.iloc[idx, :3].values.astype('int64')))
      # print("GAMEID",torch.FloatTensor(self.dataframe.iloc[idx, :3].values.astype('int64')).int())
      sample = {
          'input': result_tensor,
          'target': torch.FloatTensor(self.dataframe.iloc[idx, 8:20].values.astype('float64')),
          'play_data': torch.FloatTensor(self.dataframe.iloc[idx, 20:].values.astype('float64')),
          'ids': torch.FloatTensor(self.dataframe.iloc[idx, :3].values.astype('int64'))
          }
      return sample

In [ ]:
# train = pd.concat([X_train, y_train], axis = 1)
# test = pd.concat([X_test, y_test], axis = 1)

In [9]:
#traindataset = IntitalModelDataset(data)
#trainloader = DataLoader(traindataset, batch_size=64, shuffle=True)
testdataset = IntitalModelDataset(data)
testloader = DataLoader(testdataset, batch_size=64, shuffle=True)

Changes:
- droupout
- other common NN methods
- beef up our linear model size
- change the optimizer
- look at criterion

In [10]:
class BaseModel(nn.Module):
    def __init__(self,input_size, play_data_size, hidden_size_lstm, hidden_size_play, output_size, device):
        super(BaseModel, self).__init__()
        self.device = device

        #lstm input -> player tracking embedding
        self.lstm = nn.LSTM(input_size, hidden_size_lstm, batch_first=True).to(self.device)

        # play data -> play embedding
        self.fc = nn.Linear(play_data_size, 32).to(self.device)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(0.2)

        self.fc1 = nn.Linear(32, hidden_size_play).to(self.device)
        self.drop1 = nn.Dropout(0.2)

        #combined embeddings to final output\
        self.fc_all = nn.Linear(hidden_size_lstm + hidden_size_play, 256).to(self.device)
        self.drop2 = nn.Dropout(0.2)

        self.fc_all2 = nn.Linear(256, 128).to(self.device)
        self.drop3 = nn.Dropout(0.2)

        self.fc_all3 = nn.Linear(128, 64).to(self.device)
        self.drop4 = nn.Dropout(0.2)

        self.fc_all4 = nn.Linear(64, 12).to(self.device)

    def forward(self, lstm_data, play_data):
        # #print("x",x.shape)
        # out, _ = self.lstm(x)
        # out = self.fc(out[:, -1, :])  # Take the output from the last time step
        # #print("out",out.shape)
        # final_output = torch.softmax(out, dim = 1)

        # LSTM input
        lstm_out, _ = self.lstm(lstm_data)

        # Player tracking embedding
        play_emb = self.drop(self.relu(self.fc(play_data)))
        play_emb = self.drop1(self.relu(self.fc1(play_emb)))
        #print("PLAY EMBEDDING", play_emb)

        # Concatenate LSTM output and player tracking embedding
        combined = torch.cat((lstm_out[:, -1, :], play_emb), dim=1)
        #print("COMBINED EMBEDDING",combined)

        # Combined embeddings to final output
        out = self.drop2(self.relu(self.fc_all(combined)))
        out = self.drop3(self.relu(self.fc_all2(out)))
        out = self.drop4(self.relu(self.fc_all3(out)))
        out = self.fc_all4(out)
        #final_output = torch.softmax(out, dim = 1)
        #print("FINAL", final_output)
        return out

In [ ]:
input_size = 176
hidden_size_lstm = 128
hidden_size_play = 16
output_size = 12
play_data_size = 10
model = BaseModel(input_size, play_data_size, hidden_size_lstm, hidden_size_play, output_size, device=device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01) # Optimizer
criterion = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(10):
    running_loss = 0.0
    for batch in trainloader:
        #print(batch)
        inputs, targets, play_data, ids = batch['input'], batch['target'], batch['play_data'], batch['ids']
        optimizer.zero_grad()
        output = model(inputs.float().cuda(), play_data.cuda()).cuda()
        #print("OUTPUT", output)
        targets = targets.requires_grad_()
        loss = criterion(output.cuda(), targets.cuda())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        #if i % 100 == 99:
            #print('[%d, %5d] loss: %.3f' %
                  #(epoch + 1, i + 1, running_loss / 100))
            #running_loss = 0.0

    avg_loss = running_loss / len(trainloader)
    print('Epoch %d average loss: %.3f' % (epoch + 1, avg_loss))

Epoch 1 average loss: 2.390
Epoch 2 average loss: 2.388
Epoch 3 average loss: 2.377
Epoch 4 average loss: 2.365
Epoch 5 average loss: 2.356
Epoch 6 average loss: 2.340
Epoch 7 average loss: 2.329
Epoch 8 average loss: 2.317
Epoch 9 average loss: 2.309
Epoch 10 average loss: 2.304


In [ ]:
torch.save(model, "/content/drive/MyDrive/Home Projects/Big Data Bowl 2023/play_data_250k_all_train_new_model.pth")

In [11]:
#Reading in model
#matt_dzwil
#model = torch.load("/content/drive/MyDrive/Home Projects/Big Data Bowl 2023/play_data_250k_all_train_model.pth")

#dzwil_matt
model = torch.load("/content/drive/MyDrive/Big Data Bowl 2023/play_data_250k_all_train_model.pth")

model.eval()

BaseModel(
  (lstm): LSTM(176, 128, batch_first=True)
  (fc): Linear(in_features=10, out_features=32, bias=True)
  (relu): ReLU()
  (drop): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=32, out_features=16, bias=True)
  (drop1): Dropout(p=0.2, inplace=False)
  (fc_all): Linear(in_features=144, out_features=256, bias=True)
  (drop2): Dropout(p=0.2, inplace=False)
  (fc_all2): Linear(in_features=256, out_features=128, bias=True)
  (drop3): Dropout(p=0.2, inplace=False)
  (fc_all3): Linear(in_features=128, out_features=64, bias=True)
  (drop4): Dropout(p=0.2, inplace=False)
  (fc_all4): Linear(in_features=64, out_features=12, bias=True)
  (drop5): Dropout(p=0.2, inplace=False)
)

In [10]:
data.columns

Index(['gameId', 'playId', 'frameId', 'four', 'three', 'two', 'one', 'current',
       'tackle_1', 'tackle_2', 'tackle_3', 'tackle_4', 'tackle_5', 'tackle_6',
       'tackle_7', 'tackle_8', 'tackle_9', 'tackle_10', 'tackle_11',
       'tackle_12', 'quarter', 'down', 'yardsToGo', 'yardlineNumber',
       'totalSeconds', 'possesionScore', 'defensiveScore',
       'absoluteYardlineNumber', 'possessionWinProb', 'defensiveWinProb'],
      dtype='object')

In [12]:
final_df = pd.DataFrame()

all_targets = []
all_outputs = []
with torch.no_grad():
    for batch in testloader:
        inputs, targets, play_data, ids = batch['input'], batch['target'], batch['play_data'], batch['ids']
        outputs = model(inputs.float().cuda(), play_data.cuda()).cuda()
        out = torch.softmax(outputs, dim=1)
        columns = ["gameId","playId","frameId","tackle_1","tackle_2","tackle_3","tackle_4","tackle_5","tackle_6","tackle_7","tackle_8","tackle_9","tackle_10","tackle_11","tackle_12"]
        np_ids = ids.numpy()
        outputs_np = out.cpu().numpy()
        test = pd.DataFrame(data=np.concatenate([np_ids, outputs_np], axis=1), columns=columns)
        final_df = pd.concat([final_df, test], ignore_index=True)
        all_targets.append(targets.numpy())
        all_outputs.append(outputs.cpu().numpy())

all_targets = np.concatenate(all_targets)
all_outputs = np.concatenate(all_outputs)

# Convert to PyTorch tensors
targets_tensor = torch.tensor(all_targets)
outputs_tensor = torch.tensor(all_outputs)

# Calculate loss
total_loss = torch.nn.functional.mse_loss(outputs_tensor, targets_tensor, reduction='mean')

print("Total Loss:", total_loss.item())

final_df


Total Loss: 6017.1474609375


,gameId,playId,frameId,tackle_1,tackle_2,tackle_3,tackle_4,tackle_5,tackle_6,tackle_7,tackle_8,tackle_9,tackle_10,tackle_11,tackle_12
0,92506.0,1370.0,25.0,4.335979e-19,3.509182e-29,9.200339e-26,3.003267e-23,3.811230e-22,1.507713e-25,1.594658e-30,1.000000e+00,2.874321e-23,3.501338e-26,4.231710e-26,8.417872e-20
1,102300.0,3620.0,34.0,5.168450e-18,2.029687e-23,1.183618e-14,4.814923e-15,2.027985e-15,4.716808e-15,1.116694e-18,2.505508e-11,1.000000e+00,3.315795e-20,1.409809e-19,4.926906e-15
2,110608.0,2814.0,19.0,1.093524e-03,1.633552e-04,1.792443e-02,4.071287e-01,5.898707e-03,3.259920e-02,9.234559e-03,4.156476e-02,4.714323e-01,6.103618e-03,1.184218e-03,5.672687e-03
3,100901.0,2330.0,3.0,1.883015e-21,5.591081e-22,5.547951e-22,2.189452e-19,2.460335e-18,1.000000e+00,3.802029e-19,4.646171e-18,6.237549e-19,2.568225e-21,4.124816e-25,6.112495e-19
4,101607.0,1922.0,23.0,4.229377e-30,6.353360e-30,2.430698e-25,4.373927e-32,1.700185e-26,8.881350e-35,1.000000e+00,1.125880e-29,6.004886e-40,1.937505e-40,2.108643e-14,5.706845e-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262019,91108.0,2208.0,24.0,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
262020,102310.0,2486.0,34.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
262021,91806.0,3609.0,58.0,4.203895e-45,0.000000e+00,1.778808e-41,7.146622e-44,2.781910e-39,0.000000e+00,0.000000e+00,1.401298e-45,9.997316e-01,0.000000e+00,2.683899e-04,7.080428e-39
262022,92501.0,2469.0,12.0,6.815674e-20,2.124856e-23,8.568689e-27,5.381627e-23,3.304946e-21,3.995395e-24,5.713792e-28,2.636909e-24,1.382086e-21,1.000000e+00,7.532229e-24,2.865359e-20


# Adding in other data to perform xT analysis

In [13]:
players = pd.read_csv('/content/drive/MyDrive/Big Data Bowl 2023/data/players.csv')
data_dir = '/content/drive/MyDrive/Big Data Bowl 2023/data/'
all_ids = pd.read_csv(data_dir + "all_ids_per_frame_only_defense.csv")
all_ids = all_ids.iloc[:, :-2]
all_ids = all_ids.dropna(subset=['nflId'])
all_ids

,gameId,playId,frameId,nflId
0,2022090800,56,1,42816.0
1,2022090800,56,1,43294.0
2,2022090800,56,1,43298.0
3,2022090800,56,1,43335.0
4,2022090800,56,1,41239.0
...,...,...,...,...
5828664,2022110700,3787,44,46077.0
5828665,2022110700,3787,44,44859.0
5828666,2022110700,3787,44,53460.0
5828667,2022110700,3787,44,52627.0


In [17]:
players = pd.read_csv('/content/drive/MyDrive/Home Projects/Big Data Bowl 2023/data/players.csv')
data_dir = '/content/drive/MyDrive/Home Projects/Big Data Bowl 2023/data/'
all_ids = pd.read_csv(data_dir + "all_ids_per_frame_only_defense.csv")
all_ids = all_ids.iloc[:, :-2]
all_ids = all_ids.dropna(subset=['nflId'])
all_ids

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Home Projects/Big Data Bowl 2023/data/players.csv'

In [14]:
grouped_df = all_ids.groupby(['gameId', 'playId', 'frameId']).agg({'nflId': list}).reset_index()
grouped_df


,gameId,playId,frameId,nflId
0,2022090800,56,1,"[42816.0, 43294.0, 43298.0, 43335.0, 41239.0, ..."
1,2022090800,56,2,"[42816.0, 43294.0, 43298.0, 43335.0, 41239.0, ..."
2,2022090800,56,3,"[42816.0, 43294.0, 43298.0, 43335.0, 41239.0, ..."
3,2022090800,56,4,"[42816.0, 43294.0, 43298.0, 43335.0, 41239.0, ..."
4,2022090800,56,5,"[42816.0, 43294.0, 43298.0, 43335.0, 41239.0, ..."
...,...,...,...,...
529874,2022110700,3787,40,"[54606.0, 44828.0, 52578.0, 54541.0, 53533.0, ..."
529875,2022110700,3787,41,"[54606.0, 44828.0, 52578.0, 54541.0, 53533.0, ..."
529876,2022110700,3787,42,"[54606.0, 44828.0, 52578.0, 54541.0, 53533.0, ..."
529877,2022110700,3787,43,"[54606.0, 44828.0, 52578.0, 54541.0, 53533.0, ..."


In [15]:
def add_back_year(aFloat):
    aInt = int(aFloat)
    string = str(aInt)
    if len(string) == 5:
      final_string = "20220" + string
      return int(final_string)
    elif len(string)== 6:
      final_string = "2022" + string
      return int(final_string)
    else:
      int(final_string)

final_df['gameId'] = final_df['gameId'].apply(add_back_year)
final_df['playId'] = final_df['playId'].astype(int)
final_df['frameId'] = final_df['frameId'].astype(int)
with_players = final_df.merge(grouped_df, on = ['gameId', 'playId','frameId'], how='inner' )

In [16]:
with_players.sort_values(['gameId','playId', 'frameId'])

,gameId,playId,frameId,tackle_1,tackle_2,tackle_3,tackle_4,tackle_5,tackle_6,tackle_7,tackle_8,tackle_9,tackle_10,tackle_11,tackle_12,nflId
66334,2022090800,56,1,4.395763e-25,9.999828e-01,1.710736e-05,3.950544e-23,2.314319e-14,3.069848e-11,1.190313e-21,4.846526e-19,4.285812e-20,1.928433e-19,5.838638e-23,2.066971e-14,"[42816.0, 43294.0, 43298.0, 43335.0, 41239.0, ..."
79632,2022090800,56,2,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,"[42816.0, 43294.0, 43298.0, 43335.0, 41239.0, ..."
173834,2022090800,56,3,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,"[42816.0, 43294.0, 43298.0, 43335.0, 41239.0, ..."
210353,2022090800,56,4,0.000000e+00,1.215082e-25,1.000000e+00,1.442413e-40,1.498964e-29,2.150391e-40,1.906077e-36,2.149354e-40,5.889562e-38,1.653322e-40,5.351146e-38,6.037007e-31,"[42816.0, 43294.0, 43298.0, 43335.0, 41239.0, ..."
95168,2022090800,56,5,1.958025e-37,1.159992e-07,9.999999e-01,3.106091e-31,1.142482e-20,7.744272e-33,1.667250e-18,1.771648e-31,4.689000e-30,3.868290e-32,1.426906e-26,1.412467e-21,"[42816.0, 43294.0, 43298.0, 43335.0, 41239.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152898,2022110700,3686,72,1.972029e-35,4.203895e-45,1.961545e-31,1.071925e-34,9.369968e-32,3.879111e-30,6.050653e-40,5.671162e-29,1.000000e+00,8.518967e-39,6.097064e-39,1.477527e-29,"[54514.0, 53505.0, 44871.0, 48537.0, 43409.0, ..."
133818,2022110700,3686,73,2.063516e-37,0.000000e+00,2.385339e-33,1.367940e-36,1.645483e-33,9.595904e-32,3.354709e-42,1.751804e-30,1.000000e+00,3.954464e-41,3.342657e-41,3.510571e-31,"[54514.0, 53505.0, 44871.0, 48537.0, 43409.0, ..."
114632,2022110700,3686,74,5.386851e-28,1.237449e-35,1.878793e-24,3.080575e-26,1.073362e-24,5.410982e-23,1.579149e-30,8.620282e-22,1.000000e+00,1.979624e-30,1.069640e-30,3.475024e-23,"[54514.0, 53505.0, 44871.0, 48537.0, 43409.0, ..."
204787,2022110700,3686,75,2.515717e-15,1.782308e-19,1.878088e-12,3.222090e-12,7.869099e-13,3.046969e-11,7.069243e-15,4.679970e-10,1.000000e+00,6.513805e-16,7.394322e-17,2.129257e-12,"[54514.0, 53505.0, 44871.0, 48537.0, 43409.0, ..."


In [17]:
frame_xT_output = []

for index, row in with_players.iterrows():
    gameId = row['gameId']
    playId = row['playId']
    frameId = row['frameId']
    nfl_ids = row['nflId']

    for i, tackle_val in enumerate(row[['tackle_1', 'tackle_2', 'tackle_3', 'tackle_4', 'tackle_5', 'tackle_6', 'tackle_7', 'tackle_8', 'tackle_9', 'tackle_10', 'tackle_11', 'tackle_12']]):
       if i != 11:
        frame_xT_output.append({'nflId': nfl_ids[i], 'gameId': gameId, 'playId': playId, 'frameId': frameId,'xT': tackle_val})
       else:
        frame_xT_output.append({'nflId': 0, 'gameId': gameId, 'playId': playId, 'frameId': frameId, 'xT': tackle_val})
frame_xT_df = pd.DataFrame(frame_xT_output)

frame_xT_df['nflId'] = frame_xT_df['nflId'].astype(int)
frame_xT_df

,nflId,gameId,playId,frameId,xT
0,54469,2022092506,1370,25,4.335979e-19
1,46211,2022092506,1370,25,3.509182e-29
2,53583,2022092506,1370,25,9.200339e-26
3,44929,2022092506,1370,25,3.003267e-23
4,46204,2022092506,1370,25,3.811230e-22
...,...,...,...,...,...
3144283,52473,2022092506,614,49,0.000000e+00
3144284,45226,2022092506,614,49,0.000000e+00
3144285,52515,2022092506,614,49,0.000000e+00
3144286,43350,2022092506,614,49,0.000000e+00


In [ ]:
intial_emily = frame_xT_df.sort_values(['gameId','playId', 'frameId'])
intial_emily.to_csv('/content/drive/MyDrive/Home Projects/Big Data Bowl 2023/data/xT_by_player_frame.csv')

In [33]:
def quant(x):
    return  x.quantile(0.75)
axT_noId = with_players.groupby(['gameId', 'playId']).agg({
    'nflId': 'first',
    'tackle_1': quant,
    'tackle_2': quant,
    'tackle_3': quant,
    'tackle_4': quant,
    'tackle_5': quant,
    'tackle_6': quant,
    'tackle_7': quant,
    'tackle_8': quant,
    'tackle_9': quant,
    'tackle_10': quant,
    'tackle_11': quant,
    'tackle_12': quant
}).reset_index()

In [34]:
axT_noId

,gameId,playId,nflId,tackle_1,tackle_2,tackle_3,tackle_4,tackle_5,tackle_6,tackle_7,tackle_8,tackle_9,tackle_10,tackle_11,tackle_12
0,2022090800,56,"[42816.0, 43294.0, 43298.0, 43335.0, 41239.0, ...",0.000000e+00,7.499872e-01,2.305469e-12,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.786656e-44
1,2022090800,101,"[42816.0, 43294.0, 47862.0, 43298.0, 43335.0, ...",7.264217e-08,6.389580e-08,4.286171e-07,3.730149e-06,3.499360e-07,8.389533e-06,3.332056e-07,1.000000e+00,1.070882e-02,2.460777e-06,9.386460e-01,4.053023e-07
2,2022090800,146,"[42816.0, 43294.0, 47862.0, 43298.0, 43335.0, ...",9.571966e-30,5.040530e-25,1.334807e-23,2.354344e-25,5.169073e-22,1.479501e-27,3.181590e-30,5.403540e-11,9.877498e-23,1.000000e+00,5.575062e-08,3.680874e-23
3,2022090800,212,"[42816.0, 43294.0, 47862.0, 41239.0, 47844.0, ...",8.957194e-13,4.601054e-15,2.027077e-01,8.440733e-08,2.867935e-09,1.805496e-08,3.053849e-09,2.076149e-04,1.000000e+00,4.784636e-12,1.840930e-11,5.450825e-09
4,2022090800,236,"[42816.0, 43294.0, 43335.0, 41239.0, 47844.0, ...",1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6178,2022110700,3401,"[54606.0, 42361.0, 44828.0, 52479.0, 52578.0, ...",0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.500000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
6179,2022110700,3480,"[42361.0, 44828.0, 41363.0, 54541.0, 53533.0, ...",1.000000e+00,3.180645e-35,6.259712e-33,8.096888e-26,8.537761e-27,1.000000e+00,9.115955e-27,5.436773e-34,7.436346e-34,2.338395e-31,5.750509e-04,7.387960e-28
6180,2022110700,3581,"[54514.0, 53505.0, 44871.0, 48537.0, 46197.0, ...",4.875226e-31,2.728786e-29,6.332445e-31,2.171469e-27,8.907911e-26,1.180714e-33,7.051351e-35,5.413387e-33,5.219655e-29,2.965745e-10,1.000000e+00,8.677426e-25
6181,2022110700,3658,"[37724.0, 54514.0, 53505.0, 44871.0, 48537.0, ...",1.229017e-22,1.021715e-25,1.873018e-19,8.354509e-17,5.998803e-17,1.280684e-18,8.795325e-11,1.000000e+00,4.961540e-08,1.378594e-15,5.451719e-25,2.985908e-16


In [35]:
xT_output = []

for index, row in axT_noId.iterrows():
    gameId = row['gameId']
    playId = row['playId']
    nfl_ids = row['nflId']

    for i, tackle_val in enumerate(row[['tackle_1', 'tackle_2', 'tackle_3', 'tackle_4', 'tackle_5', 'tackle_6', 'tackle_7', 'tackle_8', 'tackle_9', 'tackle_10', 'tackle_11', 'tackle_12']]):
       if i != 11:
        xT_output.append({'nflId': nfl_ids[i], 'gameId': gameId, 'playId': playId, 'xT': tackle_val})
       else:
        xT_output.append({'nflId': 0, 'gameId': gameId, 'playId': playId, 'xT': tackle_val})
xT_df = pd.DataFrame(xT_output)

xT_df['nflId'] = xT_df['nflId'].astype(int)
xT_df

,nflId,gameId,playId,xT
0,42816,2022090800,56,0.000000e+00
1,43294,2022090800,56,7.499872e-01
2,43298,2022090800,56,2.305469e-12
3,43335,2022090800,56,0.000000e+00
4,41239,2022090800,56,0.000000e+00
...,...,...,...,...
74191,40017,2022110700,3686,1.673650e-06
74192,38607,2022110700,3686,7.436822e-01
74193,52482,2022110700,3686,9.957639e-01
74194,48027,2022110700,3686,9.935410e-07


In [36]:
total_xT = xT_df.groupby(['nflId']).agg({'xT': 'sum', 'playId': 'count'}).reset_index()
total_xT['xT'] = total_xT['xT'].round(2)
total_xT

,nflId,xT,playId
0,0,0.54,6183
1,33131,10.35,128
2,35449,9.33,61
3,35452,3.27,66
4,35454,3.23,86
...,...,...,...
842,55190,1.20,15
843,55200,0.00,5
844,55239,0.72,10
845,55240,2.00,8


In [37]:
tackles = pd.read_csv(data_dir + "tackles.csv")
tackles = tackles[tackles['playId'].isin(xT_df.playId.unique().tolist()) ]
tackles = tackles[tackles['gameId'].isin(xT_df.gameId.unique().tolist()) ]
tackles

,gameId,playId,nflId,tackle,assist,forcedFumble,pff_missedTackle
0,2022090800,101,42816,1,0,0,0
3,2022090800,646,47939,1,0,0,0
4,2022090800,818,40107,1,0,0,0
5,2022090800,1286,44976,1,0,0,0
6,2022090800,1736,44976,1,0,0,0
...,...,...,...,...,...,...,...
17420,2022092502,3252,47832,0,0,0,1
17421,2022091113,2494,43533,0,0,0,1
17422,2022092502,3510,42406,0,0,0,1
17424,2022091901,3578,42431,0,0,0,1


In [38]:
total_T = tackles.groupby(['nflId']).agg({'tackle': 'sum', 'assist': 'sum', 'playId': 'count'}).reset_index()
total_T['assist'] = total_T['assist']/2
total_T['tackle_credit'] = total_T['assist'] + total_T['tackle']
total_T

,nflId,tackle,assist,playId,tackle_credit
0,33131,7,4.5,19,11.5
1,35449,2,2.0,7,4.0
2,35452,6,3.0,13,9.0
3,35454,2,1.5,6,3.5
4,35459,23,10.0,49,33.0
...,...,...,...,...,...
783,55190,1,0.0,1,1.0
784,55200,1,0.0,1,1.0
785,55239,0,1.5,3,1.5
786,55240,2,0.5,3,2.5


In [45]:
tackles_xT = xT_df.merge(tackles, on = ['gameId','playId', 'nflId'], how='left')
tackles_xT['xT'] = tackles_xT['xT'].round(4)
tackles_xT['tackle'] = tackles_xT['tackle'].fillna(0)
tackles_xT['assist'] = tackles_xT['assist'].fillna(0)
tackles_xT['forcedFumble'] = tackles_xT['forcedFumble'].fillna(0)
tackles_xT['pff_missedTackle'] = tackles_xT['pff_missedTackle'].fillna(0)
tackles_xT

,nflId,gameId,playId,xT,tackle,assist,forcedFumble,pff_missedTackle
0,42816,2022090800,56,0.0000,0.0,0.0,0.0,0.0
1,43294,2022090800,56,0.7500,1.0,0.0,0.0,0.0
2,43298,2022090800,56,0.0000,0.0,0.0,0.0,0.0
3,43335,2022090800,56,0.0000,0.0,0.0,0.0,0.0
4,41239,2022090800,56,0.0000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
74191,40017,2022110700,3686,0.0000,0.0,0.0,0.0,0.0
74192,38607,2022110700,3686,0.7437,0.0,0.0,0.0,0.0
74193,52482,2022110700,3686,0.9958,0.0,0.0,0.0,0.0
74194,48027,2022110700,3686,0.0000,1.0,0.0,0.0,0.0


In [46]:
final_xT = tackles_xT.groupby(['nflId']).agg({'xT': 'sum', 'tackle': 'sum', 'assist': 'sum','playId': 'count'}).reset_index()
final_xT['xTDiff'] = final_xT['tackle'] + (final_xT['assist']/2) - final_xT['xT']
final_xT

,nflId,xT,tackle,assist,playId,xTDiff
0,0,0.5296,0.0,0.0,6183,-0.5296
1,33131,10.3474,2.0,7.0,128,-4.8474
2,35449,9.3295,1.0,1.0,61,-7.8295
3,35452,3.2675,5.0,3.0,66,3.2325
4,35454,3.2293,1.0,2.0,86,-1.2293
...,...,...,...,...,...,...
842,55190,1.1952,0.0,0.0,15,-1.1952
843,55200,0.0002,1.0,0.0,5,0.9998
844,55239,0.7153,0.0,3.0,10,0.7847
845,55240,2.0000,1.0,0.0,8,-1.0000


In [49]:
final_sorted = final_xT.sort_values(by='xTDiff', ascending=False)
final_sorted.to_csv('xT_table_all_testing_final.csv')
#final_sorted

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, multilabel_confusion_matrix
import numpy as np
import seaborn as sns

confusion = np.zeros((12, 12))
predictions = []
targets = []
all_output_flat = []
all_target_flat = []

# matt i moved this above so i could run my thing too
#with torch.no_grad():
    #for batch in testloader:
        #inputs, targets = batch['input'], batch['target']
        #outputs = model(inputs.float().cuda()).cpu()
        #output_np = (outputs > 0.5).int().numpy()
        #target_np = targets.int().numpy()

        # output_flat = np.argmax(output_np, axis=1)
        # target_flat = np.argmax(target_np, axis=1)

        # all_output_flat.extend(output_flat)
        # all_target_flat.extend(target_flat)

# Calculate confusion matrix for all batches
        conf_matrix = multilabel_confusion_matrix(target_np, output_np, labels=np.arange(12))

        print("Confusion Matrix:")
        print(conf_matrix)

# # Create a heatmap using seaborn and matplotlib for the combined confusion matrix
# plt.figure(figsize=(10, 8))
# sns.set(font_scale=1.2)  # Adjust the font size if needed
# sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
#             xticklabels=np.arange(11), yticklabels=np.arange(11))
# plt.xlabel("Predicted Label")
# plt.ylabel("True Label")
# plt.title("Combined Confusion Matrix")
# plt.show()

Streaming output truncated to the last 5000 lines.
 [[57  4]
  [ 0  3]]

 [[60  2]
  [ 0  2]]

 [[55  7]
  [ 0  2]]

 [[60  4]
  [ 0  0]]

 [[54  3]
  [ 0  7]]

 [[58  1]
  [ 0  5]]

 [[56  3]
  [ 0  5]]

 [[60  2]
  [ 0  2]]

 [[56  1]
  [ 0  7]]

 [[60  1]
  [ 0  3]]

 [[62  0]
  [ 0  2]]]
Confusion Matrix:
[[[54  5]
  [ 0  5]]

 [[60  2]
  [ 0  2]]

 [[62  1]
  [ 0  1]]

 [[61  1]
  [ 1  1]]

 [[60  2]
  [ 0  2]]

 [[57  2]
  [ 0  5]]

 [[56  5]
  [ 0  3]]

 [[54  5]
  [ 0  5]]

 [[59  1]
  [ 0  4]]

 [[57  3]
  [ 0  4]]

 [[55  3]
  [ 0  6]]

 [[63  1]
  [ 0  0]]]
Confusion Matrix:
[[[57  0]
  [ 0  7]]

 [[61  2]
  [ 0  1]]

 [[61  1]
  [ 0  2]]

 [[56  2]
  [ 1  5]]

 [[59  2]
  [ 0  3]]

 [[59  1]
  [ 0  4]]

 [[55  5]
  [ 0  4]]

 [[58  1]
  [ 0  5]]

 [[55  7]
  [ 0  2]]

 [[58  3]
  [ 0  3]]

 [[57  0]
  [ 0  7]]

 [[64  0]
  [ 0  0]]]
Confusion Matrix:
[[[60  2]
  [ 0  2]]

 [[56  4]
  [ 0  4]]

 [[54  2]
  [ 0  8]]

 [[57  1]
  [ 0  6]]

 [[62  0]
  [ 0  2]]

 [[60  2]
  [ 0

KeyboardInterrupt: ignored

In [ ]:
inputs

'input'

In [ ]:
#Katies Adding

test.reset_index(drop=True, inplace=True)

with torch.no_grad():
    for i, batch in enumerate(testloader):
        inputs, batch_targets = batch['input'], batch['target']
        outputs = model(inputs.float().cuda()).cpu()
        output_np = (outputs > 0.5).int().numpy()
        target_np = batch_targets.int().numpy()

        # Store predictions and targets for analysis
        predictions.extend(output_np)
        targets.extend(target_np)

In [11]:
tackle_list = pd.read_pickle('/content/drive/MyDrive/Big Data Bowl 2023/data/tackle_list.pkl')

In [12]:
tackle_comp = tackle_list.merge(data[['gameId','playId','frameId','tackle_1', 'tackle_2', 'tackle_3', 'tackle_4', 'tackle_5', 'tackle_6',
       'tackle_7', 'tackle_8', 'tackle_9', 'tackle_10', 'tackle_11', 'tackle_12']], on = ['gameId','playId','frameId'], how = "inner")

In [13]:
from sklearn.metrics import hamming_loss
tackle_list = tackle_comp['tackle_list'].tolist()
tackle_columns = [f'tackle_{i}' for i in range(1, 13)]
tackle_values = tackle_comp[tackle_columns].values.astype(float)

# Convert tackle_list to a binary matrix
tackle_list_binary = np.array([[1 if str(i) in row else 0 for i in range(1, 13)] for row in tackle_list])
tackle_list
# Convert to PyTorch tensors
tackle_list_tensor = torch.tensor(tackle_list, dtype=torch.float32)
tackle_values_tensor = torch.tensor(tackle_values, dtype=torch.float32)

# print(tackle_values_tensor)
# # # Calculate loss
loss = torch.nn.functional.mse_loss(torch.sigmoid(tackle_values_tensor), tackle_list_tensor, reduction='mean')

print(f'Loss: {loss.item()}')

Cross-Entropy Loss: 0.2599789798259735
